# 步骤三：（仅alice和bob）数据授权

这一步alice和bob需要对数据的使用进行授权，比如允许与哪些参与方的指定数据进行联合计算、允许执行的代码等，TrustFlow称之授权策略（policy）。TrustFlow提供了一套语法用于表述授权策略，您可以阅读[授权策略](../architecture/policy.md)了解更多。

我们继续以breast cancer数据集为例，alice和bob期望联合双方的数据依次进行求交、数据集拆分、树模型（XGBoost）建模、树模型（XGBoost）预测、二分类评估。为了达成这一目标，alice和bob需要分别对各自的数据进行授权。该步骤的内容alice和bob均需要分别执行。

需要授权的policy包含几个方面：
- 授权的机构ID
- 授权的算法
- 授权的数据列
- 授权的一些约束（可选）

## 选项一：仿真模式

1. 编写授权策略

我们需要在yaml配置文件（即步骤二中的alice.yaml和bob.yaml）里编写授权策略。

alice的授权示例写法如下，该策略表达了以下含义：

- `scope`: 数据的授权范围为default。（您的授权策略只会在指定的scope下生效，本教程中都将使用default作为scope的值）。
- `data_uuid`：alice将data_uuid为breast_cancer_alice作为要授权的对象。
`rules`是一个包含多条授权规则的列表，列表中每一项是一个完整的授权规则，包含以下条目：
- `rule_id`：alice为它要授权的规则取了id号为alice_rule_id_1。如果后续有删除该条规则的需求，可以根据该id号来做删除。
- `grantee_party_ids`: alice指定被他授权的人是carol，因为可以授权给多个人，所以是一个列表。
- `columns`: alice允许carol使用数据的这些列：id、mean radius、mean texture、mean perimeter、mean area、mean smoothness。
- `op_constraints`: alice允许carol执行以下计算：数据求交（`psi`）、数据拆分（`train_test_split`）、XGB训练（`xgb_train`）、XGB预测（`xgb_predict`）、二分类评估（`biclassification_eval`）。关于算子的更详细说明，可以阅读[可信APP](../architecture/apps/index.rst)。


下面的配置还需要您根据实际情况进行完善，包含：
- grantee_party_ids：请填写真实的carol机构ID（如何生成机构ID，可以阅读步骤二[第三步](step2.ipynb#第三步上传数据密钥)中的仿真模式。

```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_alice
  rules:
    - 
      # (required) str
      rule_id: alice_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean radius
        - mean texture
        - mean perimeter
        - mean area
        - mean smoothness
      # (optional) List[str]
      global_constraints:
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```

同理bob也需要修改自己的bob.yaml。我们也给出例子：

- grantee_party_ids：请填写真实的carol机构ID（如何生成机构ID，可以阅读步骤二[第三步](step2.ipynb#第三步上传数据密钥)中的仿真模式。

```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_bob
  rules:
    - 
      # (required) str
      rule_id: bob_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean compactness
        - mean concavity
        - mean concave points
        - mean symmetry
        - mean fractal dimension
        - target
      # (optional) List[str]
      global_constraints:
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```

2. 提交授权策略

alice和bob各自执行以下命令，提交授权策略到CapsuleManager。

```bash
cms --config-file alice.yaml register-data-policy
```

```bash
cms --config-file bob.yaml register-data-policy
```

## 选项二：SGX模式

### 第一步：获得可信APP的度量值

与仿真模式相比，SGX模式额外需要获取可信APP的mrenclave，前文已经涉及到相关概念，相信到此时您已经对mrenclave不再陌生。

1. 启动可信APP容器

```bash
docker run -it --name teeapps-sgx --network=host -v /dev/sgx_enclave:/dev/sgx/enclave -v /dev/sgx_provision:/dev/sgx/provision --privileged=true secretflow/teeapps-sgx-ubuntu22.04:latest bash
```

2. 配置PCCS地址

修改PCCS的配置文件/etc/sgx_default_qcnl.conf，把`pccs_url`配置为PCCS的实际部署服务地址。如果您的PCCS服务没有开启tls，把`use_secure_cert`设置为false。（您应该向carol获取此信息）。

```bash
# PCCS server address
"pccs_url": "https://localhost:8081/sgx/certification/v4/",

# To accept insecure HTTPS certificate, set this option to FALSE
"use_secure_cert": false

```

把/etc/sgx_default_qcnl.conf复制到occlum的image中

```bash
cp /etc/sgx_default_qcnl.conf /home/teeapp/occlum/occlum_instance/image/etc/
```

3. 构建可信APP
您首先需要生成私钥，然后使用以下命令构建occlum。生成私钥可以参考下列脚本，生成的私钥保存在当前目录的private_key.pem。请妥善保存您的私钥，不要泄露给其他人。

```bash
openssl genrsa -3 -out private_key.pem 3072
```

生成公私钥后，使用私钥构建occlum。

```bash
occlum build -f --sign-key private_key.pem
```

4. 获取mrenclave

执行下列命令可以获得可信APP的mrenclave，输出内容为一串小写的十六进制字符串，您可以保存下来，后续步骤会使用到。

```bash
occlum print mrenclave
```

### 第二步：数据授权

1. 编写授权策略

我们需要在yaml配置文件（即步骤二中的alice.yaml和bob.yaml）里编写授权策略。

alice的授权示例写法如下，该策略表达了以下含义：


- `scope`: 数据的授权范围为default。（您的授权策略只会在指定的scope下生效，本教程中都将使用default作为scope的值）。
- `data_uuid`：alice将data_uuid为breast_cancer_alice作为要授权的对象。
- `global_constraints`: 限制可信APP所在的TEE平台类型为sgx；限制可信APP的mrenclave。
`rules`是一个包含多条授权规则的列表，列表中每一项是一个完整的授权规则，包含以下条目：
- `rule_id`：alice为它要授权的规则取了id号为alice_rule_id_1。如果后续有删除该条规则的需求，可以根据该id号来做删除。
- `grantee_party_ids`: alice指定被他授权的人是carol，因为可以授权给多个人，所以是一个列表。
- `columns`: alice允许carol使用数据的这些列：id、mean radius、mean texture、mean perimeter、mean area、mean smoothness。
- `op_constraints`: alice允许carol执行以下计算：数据求交（`psi`）、数据拆分（`train_test_split`）、XGB训练（`xgb_train`）、XGB预测（`xgb_predict`）、二分类评估（`biclassification_eval`）。关于算子的更详细说明，可以阅读[可信APP](../architecture/apps/index.rst)。

下面的配置还需要您根据实际情况进行完善，包含：

- grantee_party_ids: 请填写真实的carol机构ID（如何生成机构ID，可以阅读步骤二[第三步](step2.ipynb#第三步上传数据密钥)中的仿真模式。
- r.env.tee.sgx.mr_enclave: 填写上一步所获得的可信APP mrenclave（注意这一步的mrenclave要使用小写）。

```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_alice
  rules:
    - 
      # (required) str
      rule_id: alice_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean radius
        - mean texture
        - mean perimeter
        - mean area
        - mean smoothness
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="sgx"
        - r.env.tee.sgx.mr_enclave=="xxxx"  # xxxx 为全小写
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```

同理bob也需要修改自己的bob.yaml。我们也给出例子：

```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_bob
  rules:
    - 
      # (required) str
      rule_id: bob_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean compactness
        - mean concavity
        - mean concave points
        - mean symmetry
        - mean fractal dimension
        - target
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="sgx"
        - r.env.tee.sgx.mr_enclave=="xxxx"  # xxxx 为全小写
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```

2. 提交授权策略

alice和bob各自执行以下命令，提交授权策略到CapsuleManager。

```bash
cms --config-file alice.yaml register-data-policy
```

```bash
cms --config-file bob.yaml register-data-policy
```

## 选项三：TDX模式

### 第一步：获取可信APP的度量值
目前暂无简易工具可以获取度量值，可以跳过该步骤。

### 第二步：数据授权

1. 编写授权策略

我们需要在yaml配置文件（即步骤二中的alice.yaml和bob.yaml）里编写授权策略。

alice的授权示例写法如下，该策略表达了以下含义：

- `scope`: 数据的授权范围为default。（您的授权策略只会在指定的scope下生效，本教程中都将使用default作为scope的值）。
- `data_uuid`：alice将data_uuid为breast_cancer_alice作为要授权的对象。
- `global_constraints`: 限制可信APP所在的TEE平台类型为tdx。
`rules`是一个包含多条授权规则的列表，列表中每一项是一个完整的授权规则，包含以下条目：
- `rule_id`：alice为它要授权的规则取了id号为alice_rule_id_1。如果后续有删除该条规则的需求，可以根据该id号来做删除。
- `grantee_party_ids`: alice指定被他授权的人是carol，因为可以授权给多个人，所以是一个列表。
- `columns`: alice允许carol使用数据的这些列：id、mean radius、mean texture、mean perimeter、mean area、mean smoothness。
- `op_constraints`: alice允许carol执行以下计算：数据求交（`psi`）、数据拆分（`train_test_split`）、XGB训练（`xgb_train`）、XGB预测（`xgb_predict`）、二分类评估（`biclassification_eval`）。关于算子的更详细说明，可以阅读[可信APP](../architecture/apps/index.rst)。

下面的配置还需要您根据实际情况进行完善，包含：

- grantee_party_ids：请填写真实的carol机构ID（如何生成机构ID，可以阅读步骤二[第三步](step2.ipynb#第三步上传数据密钥)中的仿真模式。
```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_alice
  rules:
    - 
      # (required) str
      rule_id: alice_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean radius
        - mean texture
        - mean perimeter
        - mean area
        - mean smoothness
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="tdx"
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```



同理bob也需要修改自己的bob.yaml。我们也给出例子：
```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_bob
  rules:
    - 
      # (required) str
      rule_id: bob_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean compactness
        - mean concavity
        - mean concave points
        - mean symmetry
        - mean fractal dimension
        - target
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="tdx"
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```


2. 提交授权策略

alice和bob各自执行以下命令，提交授权策略到CapsuleManager。

```bash
cms --config-file alice.yaml register-data-policy
```

```bash
cms --config-file bob.yaml register-data-policy
```

## 选项四：CSV模式

### 第一步：获取CSV VM的度量值

目前暂无简易工具可以获取度量值，可以跳过该步骤。

### 第二步：数据授权

1. 编写授权策略

我们需要在yaml配置文件（即步骤二中的alice.yaml和bob.yaml）里编写授权策略。

alice的授权示例写法如下，该策略表达了以下含义：

- `scope`: 数据的授权范围为default。（您的授权策略只会在指定的scope下生效，本教程中都将使用default作为scope的值）。
- `data_uuid`：alice将data_uuid为breast_cancer_alice作为要授权的对象。
- `global_constraints`: 限制可信APP所在的TEE平台类型为csv。
`rules`是一个包含多条授权规则的列表，列表中每一项是一个完整的授权规则，包含以下条目：
- `rule_id`：alice为它要授权的规则取了id号为alice_rule_id_1。如果后续有删除该条规则的需求，可以根据该id号来做删除。
- `grantee_party_ids`: alice指定被他授权的人是carol，因为可以授权给多个人，所以是一个列表。
- `columns`: alice允许carol使用数据的这些列：id、mean radius、mean texture、mean perimeter、mean area、mean smoothness。
- `op_constraints`: alice允许carol执行以下计算：数据求交（`psi`）、数据拆分（`train_test_split`）、XGB训练（`xgb_train`）、XGB预测（`xgb_predict`）、二分类评估（`biclassification_eval`）。关于算子的更详细说明，可以阅读[可信APP](../architecture/apps/index.rst)。

下面的配置还需要您根据实际情况进行完善，包含：

- grantee_party_ids：请填写真实的carol机构ID（如何生成机构ID，可以阅读步骤二[第三步](step2.ipynb#第三步上传数据密钥)中的仿真模式。

```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_alice
  rules:
    - 
      # (required) str
      rule_id: alice_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean radius
        - mean texture
        - mean perimeter
        - mean area
        - mean smoothness
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="csv"
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```

同理bob也需要修改自己的bob.yaml。我们也给出例子：
```yaml
register_data_policy:
  # (required) str
  scope: default
  # (required) str
  data_uuid: breast_cancer_bob
  rules:
    - 
      # (required) str
      rule_id: bob_rule_id_1
      # (required) List[str]
      grantee_party_ids:
        - xxxx
      # (required) List[str]
      columns:
        - id
        - mean compactness
        - mean concavity
        - mean concave points
        - mean symmetry
        - mean fractal dimension
        - target
      # (optional) List[str]
      global_constraints:
        - r.env.tee.platform=="csv"
      # (required) List[dict]
      op_constraints:
        - 
          # (required) str
          op_name: psi
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: train_test_split
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_train
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: xgb_predict
          # (optional) List[str]
          constraints:
        -
          # (required) str
          op_name: biclassification_eval
          # (optional) List[str]
          constraints:
```


2. 提交授权策略

alice和bob各自执行以下命令，提交授权策略到CapsuleManager。

```bash
cms --config-file alice.yaml register-data-policy
```

```bash
cms --config-file bob.yaml register-data-policy
```

## （可选）自定义授权策略

在快速上手的示例之外，您可以探索更多的授权策略，根据自己的需求对数据进行授权，参见[授权策略](../architecture/policy.md)。